In [1]:
import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc_old_segments")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc_old_segments/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
Using custom data configuration corpus-48c43d9b917d9e96
Reusing dataset text (/Users/Gianni/.cache/huggingface/datasets/text/corpus-48c43d9b917d9e96/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|██████████| 3/3 [00:00<00:00, 274.44it/s]
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc_old_segments/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further traini

In [3]:
import tensorflow as tf

In [8]:
semiotic.syntagmatic.save_tokenizer()

In [7]:
sent = semiotic.corpus.train["text"][100]
# sent = "Whenever it is possible bring your own lunch to the office"
sent_tokenized = semiotic.syntagmatic.bert_tokenizer.decode(semiotic.syntagmatic.bert_tokenizer(sent)["input_ids"])
print(sent)
print(sent_tokenized)

relative abund ancesof other element sare low resulting in low mean relative abund ances varying between 170 and 4 40 percent table 23
relative abund ancesof other element sare low resulting in low mean relative abund ances varying between 170 and 4 40 percent table 23


In [43]:
sent_len = len(sent.split())
sent_mask = [" ".join([t if i!=n else "[MASK]" for i,t in enumerate(sent.split())]) for n in range(sent_len)]

In [44]:
input = semiotic.paradigmatic.bert_tokenizer(sent_mask)
outputs = semiotic.paradigmatic.model(input["input_ids"])

# input = semiotic.paradigmatic.hf_tokenizer(sent_mask, return_tensors="tf")
# outputs = semiotic.paradigmatic.hf_model(input)

# Gather all the masked logits in a batch, by taking the row diagonal of the 3D tensor

parad_logits = tf.gather_nd(outputs.logits, indices=[[i,i] for i in range(sent_len)])
# parad_logits = tf.gather_nd(outputs.logits[:,1:-1], indices=[[i,i] for i in range(sent_len)])

In [45]:
# input = semiotic.paradigmatic.bert_tokenizer(sent)
# outputs = semiotic.paradigmatic.model(input["input_ids"])
# parad_logits = outputs.logits[0]

In [46]:
semiotic.paradigmatic.tokenizer.decode(input["input_ids"][1])

'he [MASK] had [UNK] in [UNK] but hadbeen unableto visit them because he believed he wouldbe [UNK]'

In [47]:
logits_positif = tf.nn.relu(parad_logits)
probs, norms = tf.linalg.normalize(logits_positif, ord=1, axis=1)

# Maybe its cheaper to compute top k for k = non_zero, which would differ from row to row, and hence not yeld a tensor. To be tested.

non_zeroes = tf.math.count_nonzero(probs,axis=1)
max_non_zeroes = max(non_zeroes).numpy()

parad_data = tf.math.top_k(probs, k = max_non_zeroes, sorted=True)
parad_tokens = [semiotic.syntagmatic.tokenizer.decode(parad.numpy()[:parad_len]).split() for parad, parad_len in zip(parad_data.indices, non_zeroes)]

# parad_tokens = [semiotic.paradigmatic.hf_tokenizer.decode(parad.numpy()[:parad_len]).split() for parad, parad_len in zip(parad_data.indices, non_zeroes)]

sent_tokenized = semiotic.syntagmatic.bert_tokenizer.decode(semiotic.syntagmatic.bert_tokenizer(sent)["input_ids"])
print(sent_tokenized)
slg.util.df([p[:20] for p in parad_tokens], keys = sent_tokenized.split())

he still had [UNK] in [UNK] but hadbeen unableto visit them because he believed he wouldbe [UNK]


,he,still,had,[UNK],in,[UNK],but,hadbeen,unableto,visit,them,because,he,believed,he,wouldbe,[UNK]
0,he,still,had,children,to,parliament,and,was,ableto,kill,him,because,he,feared,he,wouldbe,dead
1,hehad,already,kept,aroom,for,america,but,hadbeen,allowedto,leave,them,and,she,believed,she,mightbe,ill
2,she,certainly,got,one,s,india,buthe,hewas,unableto,help,it,after,hehad,admitted,they,shouldbe,athome
3,they,too,remained,money,his,ireland,sand,were,willingto,marry,london,until,hewas,believedthat,thathe,wouldnt,there
4,hed,nolonger,didnt,trouble,with,trouble,thatthey,hehadbeen,forcedto,meet,england,eventhough,shehad,toldme,this,couldbe,married
5,hes,now,felt,him,in,manchester,whenthey,shewas,goingto,join,her,before,hed,claimed,it,musthave,here
6,we,only,left,lunch,and,italy,sbut,wasstill,expectedto,visit,britain,while,hehadbeen,believes,i,nolonger,happy
7,shehad,hardly,found,ajob,from,office,aand,theywere,tryingto,treat,me,although,itwas,and,thatthey,were,injured
8,i,always,lost,aword,ed,public,becausethey,wewere,to,buy,hospital,so,they,suggested,nothing,wouldhave,wrong
9,hehas,even,met,acar,of,control,or,shehadbeen,enoughto,sell,thehouse,though,shewas,said,everything,always,ready


In [34]:
max_parad = 100 # max_non_zeroes
slg.util.plot_scatter_line(list(range(max_parad)),parad_data.values[0],sent.split()[0], add_trace=[(list(range(max_parad)), v, t) for v, t in zip(parad_data.values[1:],sent.split()[1:])])

In [21]:
non_zeroes

<tf.Tensor: shape=(16,), dtype=int64, numpy=
array([ 444,  824,  505, 1134,  999,  700,  984,  396,  833,  404,  817,
        502,  760,  725,  398,  559])>